<a href="https://colab.research.google.com/github/tejshahi/data_fusion/blob/master/RF_Train_6band_(Training_only).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Necessary packages

In [ ]:
from osgeo import gdal_array
from osgeo import gdal
import numpy as np
filepath= "content/drive/My Drive/dataset_for_fusion/ls_sa_ndvi_training_set_c.dat" # Path to training data file
raster = gdal.Open(filepath)
rasterArray = raster.ReadAsArray()
b1 = rasterArray[0,0:2661,0:3395]
b2 = rasterArray[1,0:2661,0:3395]
b3 = rasterArray[2,0:2661,0:3395]
b4 = rasterArray[3,0:2661,0:3395]
b5 = rasterArray[4,0:2661,0:3395]
b6 = rasterArray[5,0:2661,0:3395]

In [ ]:
rasterArray.shape

(6, 2661, 3395)

In [ ]:
b11=np.reshape(b1,(b1.shape[0]*b1.shape[1],))
b22=np.reshape(b2,(b2.shape[0]*b2.shape[1],))
b33=np.reshape(b3,(b3.shape[0]*b3.shape[1],))
b44=np.reshape(b4,(b4.shape[0]*b4.shape[1],))
b55=np.reshape(b5,(b5.shape[0]*b5.shape[1],))
b66=np.reshape(b6,(b6.shape[0]*b6.shape[1],))

In [ ]:
train_x=np.stack((b11,b33,b44,b55,b66),axis=-1)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 1, stop = 20, num = 5)]
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 10, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1]
# Method of selecting samples for training each tree
bootstrap = [True]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
train_y=b22
train_y.shape

(9034095,)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
import time
from sklearn.ensemble import RandomForestRegressor
train_s_time=time.clock()
rf=RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions=random_grid, n_iter=3, cv = 2, verbose=2, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_x, train_y)
print('Training time'+str(time.clock()-train_s_time)+'secs')

In [ ]:
from joblib import dump, load
dump(rf_random.best_estimator_, 'rf_six_band.joblib') 

In [ ]:
#to calculate training time
#initialze the random forest regressor with best parameters
import time
from sklearn.ensemble import RandomForestRegressor
train_s_time=time.clock()
rf=RandomForestRegressor(n_estimators=40, max_depth=7, n_jobs=-1)
rf.fit(train_x,train_y)
print('Training time'+str(time.clock()-train_s_time)+'secs')

Training time605.3757942secs
